# Mapping Company Rules to Agent Constraints

## The Challenge

Every company has rules:
- *"Payments over $50 require manager approval"*
- *"No data exports after 6pm"*
- *"Only delete non-production resources"*

But agents need **technical constraints** they can't violate.

This notebook shows how Relay bridges that gap.

## The Process

```
Business Rule  →  YAML Policy  →  Rego Code  →  Enforcement
(Human language)  (Structured)   (Machine logic) (Automatic)
```

## Example 1: Financial Policies

### Business Rules (From CFO Email)

> *Dear Team,*
> 
> *Effective immediately, all autonomous agents must follow these spending rules:*
> *1. Single payments cannot exceed $50*
> *2. Refunds up to $100 are automatic*
> *3. Staging environment has no limits (for testing)*
> *4. All production transactions require audit logs*
> 
> *— Sarah, CFO*

### Step 1: Translate to YAML Policy

In [1]:
yaml_policy = """
version: "1.0"
package: "relay.policies.main"

policies:
  - name: "spending_limits"
    description: "CFO mandate: payments under $50"
    rules:
      - id: "small_payments_allowed"
        condition:
          provider: "stripe"
          method: "create_payment"
          parameter_constraints:
            amount:
              max: 5000  # $50.00 in cents
        action: "allow"

      - id: "large_payments_denied"
        condition:
          provider: "stripe"
          method: "create_payment"
          parameter_constraints:
            amount:
              min: 5000  # Over $50.00
        action: "deny"
        reason: "Payment exceeds $50.00 limit (CFO policy)"

  - name: "refund_policy"
    description: "Auto-approve refunds under $100"
    rules:
      - id: "small_refunds_allowed"
        condition:
          provider: "stripe"
          method: "create_refund"
          parameter_constraints:
            amount:
              max: 10000  # $100.00 in cents
        action: "allow"

  - name: "environment_exceptions"
    description: "Staging has no limits"
    rules:
      - id: "staging_unrestricted"
        condition:
          environment: "staging"
          provider: "stripe"
        action: "allow"
"""

print("📋 YAML Policy (Human-Readable)")
print("="*70)
print(yaml_policy)
print("\n✅ This YAML is version-controlled in your repo")
print("✅ CFO can review and approve changes")
print("✅ Automatically compiles to Rego for enforcement")

📋 YAML Policy (Human-Readable)

version: "1.0"
package: "relay.policies.main"

policies:
  - name: "spending_limits"
    description: "CFO mandate: payments under $50"
    rules:
      - id: "small_payments_allowed"
        condition:
          provider: "stripe"
          method: "create_payment"
          parameter_constraints:
            amount:
              max: 5000  # $50.00 in cents
        action: "allow"

      - id: "large_payments_denied"
        condition:
          provider: "stripe"
          method: "create_payment"
          parameter_constraints:
            amount:
              min: 5000  # Over $50.00
        action: "deny"
        reason: "Payment exceeds $50.00 limit (CFO policy)"

  - name: "refund_policy"
    description: "Auto-approve refunds under $100"
    rules:
      - id: "small_refunds_allowed"
        condition:
          provider: "stripe"
          method: "create_refund"
          parameter_constraints:
            amount:
              max: 10000  

### Step 2: Test the Policy

In [2]:
import sys
sys.path.insert(0, '../')
import requests

def test_business_rule(scenario, manifest_data):
    """Test a business rule scenario."""
    print(f"\n{'='*70}")
    print(f"Scenario: {scenario}")
    print(f"{'='*70}\n")
    
    response = requests.post(
        'http://localhost:8181/v1/data/relay/policies/main',
        json={'input': manifest_data}
    )
    
    result = response.json()['result']
    approved = result.get('allow', False)
    
    print(f"Business Rule: {scenario}")
    print(f"Decision: {'✅ APPROVED' if approved else '❌ DENIED'}")
    if not approved:
        print(f"Reason: {result.get('reason', 'N/A')}")
    
    return approved

# Test Scenario 1: Small payment (should pass)
test_business_rule(
    "Agent wants to pay $40 to vendor",
    {
        "action": {
            "provider": "stripe",
            "method": "create_payment",
            "parameters": {"amount": 4000}
        },
        "environment": "production"
    }
)

# Test Scenario 2: Large payment (should fail)
test_business_rule(
    "Agent wants to pay $75 to vendor",
    {
        "action": {
            "provider": "stripe",
            "method": "create_payment",
            "parameters": {"amount": 7500}
        },
        "environment": "production"
    }
)

# Test Scenario 3: Large refund (should fail)
test_business_rule(
    "Agent wants to refund $150",
    {
        "action": {
            "provider": "stripe",
            "method": "create_refund",
            "parameters": {"amount": 15000}
        },
        "environment": "production"
    }
)

# Test Scenario 4: Staging exception (should pass)
test_business_rule(
    "Agent wants to pay $200 in STAGING",
    {
        "action": {
            "provider": "stripe",
            "method": "create_payment",
            "parameters": {"amount": 20000}
        },
        "environment": "staging"
    }
)


Scenario: Agent wants to pay $40 to vendor

Business Rule: Agent wants to pay $40 to vendor
Decision: ✅ APPROVED

Scenario: Agent wants to pay $75 to vendor

Business Rule: Agent wants to pay $75 to vendor
Decision: ❌ DENIED
Reason: Payment amount exceeds $50.00 limit

Scenario: Agent wants to refund $150

Business Rule: Agent wants to refund $150
Decision: ❌ DENIED
Reason: Refund amount exceeds $100.00 limit

Scenario: Agent wants to pay $200 in STAGING

Business Rule: Agent wants to pay $200 in STAGING
Decision: ✅ APPROVED


True

## Example 2: Infrastructure Policies

### Business Rules (From CTO)

> *Team,*
> 
> *Our DevOps agents need these safety rules:*
> *1. Never delete production databases*
> *2. Resource creation limited to $100/hour*
> *3. Only specific engineers can modify security groups*
> *4. All destructive operations require 2-minute delay*
> 
> *— Mike, CTO*

In [3]:
infrastructure_policy = """
version: "1.0"
package: "relay.policies.infrastructure"

policies:
  - name: "database_protection"
    description: "Never delete production databases"
    rules:
      - id: "block_prod_db_deletion"
        condition:
          provider: "aws"
          method: "delete_database"
          parameter_constraints:
            environment:
              equals: "production"
        action: "deny"
        reason: "Cannot delete production databases (CTO policy)"

      - id: "allow_dev_db_deletion"
        condition:
          provider: "aws"
          method: "delete_database"
          parameter_constraints:
            environment:
              in: ["development", "staging"]
        action: "allow"

  - name: "cost_controls"
    description: "Resource creation budget"
    rules:
      - id: "hourly_spending_limit"
        condition:
          provider: "aws"
          method: "create_resource"
          parameter_constraints:
            estimated_cost:
              max: 10000  # $100 in cents
        action: "allow"

  - name: "security_restrictions"
    description: "Authorized engineers only"
    rules:
      - id: "security_group_changes"
        condition:
          provider: "aws"
          method: "modify_security_group"
          agent_id: "security-engineer-001"  # Whitelist
        action: "allow"

      - id: "block_unauthorized_security_changes"
        condition:
          provider: "aws"
          method: "modify_security_group"
        action: "deny"
        reason: "Only authorized security engineers can modify security groups"
"""

print("🔧 Infrastructure Policy (YAML)")
print("="*70)
print(infrastructure_policy)
print("\n✅ Protects critical infrastructure")
print("✅ Enforces cost controls")
print("✅ Access control built-in")

🔧 Infrastructure Policy (YAML)

version: "1.0"
package: "relay.policies.infrastructure"

policies:
  - name: "database_protection"
    description: "Never delete production databases"
    rules:
      - id: "block_prod_db_deletion"
        condition:
          provider: "aws"
          method: "delete_database"
          parameter_constraints:
            environment:
              equals: "production"
        action: "deny"
        reason: "Cannot delete production databases (CTO policy)"

      - id: "allow_dev_db_deletion"
        condition:
          provider: "aws"
          method: "delete_database"
          parameter_constraints:
            environment:
              in: ["development", "staging"]
        action: "allow"

  - name: "cost_controls"
    description: "Resource creation budget"
    rules:
      - id: "hourly_spending_limit"
        condition:
          provider: "aws"
          method: "create_resource"
          parameter_constraints:
            estimated_cost:


## Example 3: Data Protection Policies

### Business Rules (From Legal/Compliance)

> *All,*
> 
> *GDPR and SOC2 require these data policies:*
> *1. No PII exports outside working hours (9am-6pm)*
> *2. Customer data queries limited to 100 records*
> *3. Data deletion requires user consent token*
> *4. All data access must be logged with justification*
> 
> *— Legal Team*

In [4]:
from datetime import datetime

data_policy = """
version: "1.0"
package: "relay.policies.data"

policies:
  - name: "time_based_restrictions"
    description: "PII exports only during business hours"
    # Note: Time-based policies require policy engine support
    # This is a simplified example

  - name: "query_limits"
    description: "Limit data extraction"
    rules:
      - id: "max_100_records"
        condition:
          provider: "database"
          method: "query_customers"
          parameter_constraints:
            limit:
              max: 100
        action: "allow"

      - id: "block_large_queries"
        condition:
          provider: "database"
          method: "query_customers"
          parameter_constraints:
            limit:
              min: 100
        action: "deny"
        reason: "Query limit exceeded (max 100 records for compliance)"

  - name: "deletion_consent"
    description: "Require consent for data deletion"
    rules:
      - id: "deletion_requires_token"
        # This would check for a consent_token parameter
        # In the manifest justification or parameters
"""

print("🔒 Data Protection Policy (YAML)")
print("="*70)
print(data_policy)
print("\n✅ GDPR compliant")
print("✅ SOC2 audit ready")
print("✅ Automatic enforcement")

🔒 Data Protection Policy (YAML)

version: "1.0"
package: "relay.policies.data"

policies:
  - name: "time_based_restrictions"
    description: "PII exports only during business hours"
    # Note: Time-based policies require policy engine support
    # This is a simplified example

  - name: "query_limits"
    description: "Limit data extraction"
    rules:
      - id: "max_100_records"
        condition:
          provider: "database"
          method: "query_customers"
          parameter_constraints:
            limit:
              max: 100
        action: "allow"

      - id: "block_large_queries"
        condition:
          provider: "database"
          method: "query_customers"
          parameter_constraints:
            limit:
              min: 100
        action: "deny"
        reason: "Query limit exceeded (max 100 records for compliance)"

  - name: "deletion_consent"
    description: "Require consent for data deletion"
    rules:
      - id: "deletion_requires_token"
   

## Mapping Process: Business Rule → Technical Constraint

### Step-by-Step Translation

In [5]:
print("""
================================================================================
TRANSLATION GUIDE: Business Rules → Relay Policies
================================================================================

Step 1: Identify the Rule Type
─────────────────────────────────────────────────────

Financial Rules:
  "No payments over $X"           → amount.max constraint
  "Spending limit per Y"          → rate limiting policy
  "Require approval for $Z+"      → workflow policy

Access Control:
  "Only engineers can deploy"     → agent_id whitelist
  "Prod access restricted"        → environment constraint
  "Manager approval required"     → approval workflow

Safety Rules:
  "Never delete production"       → environment deny rule
  "Read-only access"              → method allowlist
  "No destructive operations"     → provider/method deny

Compliance:
  "Log all data access"           → audit enforcement
  "PII handling restrictions"     → parameter constraints
  "Retention policy limits"       → time-based rules


Step 2: Map to YAML Structure
─────────────────────────────────────────────────────

Business Rule:
  "Agents cannot delete production databases"

YAML Mapping:
  policies:
    - name: "production_protection"
      rules:
        - id: "block_prod_deletion"
          condition:
            provider: "aws"              # Which service
            method: "delete_database"     # Which action
            environment: "production"     # Which environment
          action: "deny"                  # Allow or deny
          reason: "Cannot delete production databases"


Step 3: Add Constraints
─────────────────────────────────────────────────────

Numeric Constraints:
  amount.max: 5000        # Maximum value
  amount.min: 100         # Minimum value
  count.max: 100          # Maximum count

String Constraints:
  environment.equals: "production"
  region.in: ["us-east-1", "us-west-2"]
  user_id.matches: "^admin-.*$"

Boolean Constraints:
  require_mfa: true
  allow_external: false


Step 4: Test & Validate
─────────────────────────────────────────────────────

1. Write policy in YAML
2. Compile to Rego
3. Test with sample manifests
4. Verify both allow and deny cases
5. Deploy to staging first
6. Monitor audit logs
7. Promote to production


Step 5: Version Control
─────────────────────────────────────────────────────

Git Workflow:
  1. Create branch: git checkout -b policy/spending-limits
  2. Edit YAML: policies/finance.yaml
  3. Test locally: python policy-compiler/compiler.py
  4. Commit: git commit -m "Add $50 spending limit"
  5. PR Review: Get CFO/Legal approval
  6. Merge: Deploy automatically via CI/CD
  7. Rollback: git revert if issues

================================================================================
""")


TRANSLATION GUIDE: Business Rules → Relay Policies

Step 1: Identify the Rule Type
─────────────────────────────────────────────────────

Financial Rules:
  "No payments over $X"           → amount.max constraint
  "Spending limit per Y"          → rate limiting policy
  "Require approval for $Z+"      → workflow policy

Access Control:
  "Only engineers can deploy"     → agent_id whitelist
  "Prod access restricted"        → environment constraint
  "Manager approval required"     → approval workflow

Safety Rules:
  "Never delete production"       → environment deny rule
  "Read-only access"              → method allowlist
  "No destructive operations"     → provider/method deny

Compliance:
  "Log all data access"           → audit enforcement
  "PII handling restrictions"     → parameter constraints
  "Retention policy limits"       → time-based rules


Step 2: Map to YAML Structure
─────────────────────────────────────────────────────

Business Rule:
  "Agents cannot delete produ

## Real-World Policy Library

In [6]:
policy_library = {
    "E-Commerce": [
        "Single refunds under $100: auto-approve",
        "Discount codes: max 20% off",
        "Inventory allocation: preserve 10% buffer",
        "Price changes: max 5% deviation from base"
    ],
    "SaaS Platform": [
        "Account upgrades: auto-approve up to $500/month",
        "License allocation: max 1000 per customer",
        "API quota adjustments: require approval",
        "Downgrade requests: execute with 30-day notice"
    ],
    "Healthcare": [
        "Patient data access: audit all queries",
        "Prescription refills: verify insurance",
        "Appointment scheduling: working hours only",
        "Data exports: require compliance officer"
    ],
    "Financial Services": [
        "Wire transfers: two-factor auth required",
        "Account closures: 7-day waiting period",
        "Credit limit increases: risk assessment",
        "International transfers: compliance check"
    ],
    "DevOps": [
        "Production deployments: during maintenance windows",
        "Resource deletion: require confirmation",
        "Security group changes: senior engineers only",
        "Database migrations: backup verification"
    ]
}

print("📚 Policy Library by Industry\n")
print("="*70)

for industry, rules in policy_library.items():
    print(f"\n{industry}:")
    for rule in rules:
        print(f"  • {rule}")

print("\n" + "="*70)
print("\n💡 All of these can be expressed as Relay policies")
print("💡 Each rule becomes deterministic, enforceable code")
print("💡 Business stakeholders can review YAML directly")

📚 Policy Library by Industry


E-Commerce:
  • Single refunds under $100: auto-approve
  • Discount codes: max 20% off
  • Inventory allocation: preserve 10% buffer
  • Price changes: max 5% deviation from base

SaaS Platform:
  • Account upgrades: auto-approve up to $500/month
  • License allocation: max 1000 per customer
  • API quota adjustments: require approval
  • Downgrade requests: execute with 30-day notice

Healthcare:
  • Patient data access: audit all queries
  • Prescription refills: verify insurance
  • Appointment scheduling: working hours only
  • Data exports: require compliance officer

Financial Services:
  • Wire transfers: two-factor auth required
  • Account closures: 7-day waiting period
  • Credit limit increases: risk assessment
  • International transfers: compliance check

DevOps:
  • Production deployments: during maintenance windows
  • Resource deletion: require confirmation
  • Security group changes: senior engineers only
  • Database migrations: backup 

## Key Benefits of This Approach

### 1. Business Stakeholder Friendly
- CFO can read and approve YAML policies
- Legal team can verify compliance
- No need to understand Rego or code

### 2. Version Controlled
- Git history shows policy evolution
- Pull requests for policy changes
- Rollback capability
- Blame/audit trail built-in

### 3. Automatic Compilation
- YAML → Rego transformation
- Syntax validation
- Test generation
- No manual coding

### 4. Environment Parity
- Same policies across dev/staging/prod
- No configuration drift
- Consistent enforcement

### 5. Compliance Ready
- SOC2: Policy-as-code evidence
- GDPR: Data handling rules
- PCI: Payment restrictions
- HIPAA: Patient data protection

## Next Steps

1. ✅ Review your company's informal rules (emails, wikis, docs)
2. ✅ Translate top 5 critical rules to YAML
3. ✅ Test with realistic scenarios
4. ✅ Deploy to staging and monitor
5. ✅ Roll out to production gradually

See `05_policy_testing.ipynb` for testing strategies.